In [1]:
import os
import numpy as np


from astropy.time import Time
from astropy.stats import mad_std
import astropy.units as u
from astropy.coordinates import (SkyCoord, EarthLocation, AltAz, get_sun, get_body)

from astropy.io import fits

from ffs import FFS, ffs_make_segments



path = "/work/vela/oca/fits/zb08/raw"
#path = "/work/vela/oca/fits/zb08/processed-ofp/science"
noce = [d for d in os.listdir(path) if (os.path.isdir(path+"/"+d) and not os.path.islink(path+"/"+d) )]

noce = sorted(noce)

print(noce)

['0088', '0091', '0095', '0096', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105', '0106', '0108', '0112', '0113', '0114', '0116', '0117', '0119', '0154', '0156', '0157', '0161', '0163', '0164', '0165', '0166', '0167', '0168', '0169', '0170', '0171', '0172', '0173', '0174', '0175', '0176', '0177', '0178', '0179', '0180', '0181', '0182', '0183', '0184', '0185', '0186', '0187', '0188', '0189', '0190', '0191', '0192', '0193', '0194', '0195', '0196', '0197', '0198', '0199', '0200', '0201', '0202', '0203', '0204', '0205', '0206', '0208', '0209', '0210', '0211', '0214', '0215', '0216', '0217', '0219', '0220', '0221', '0222', '0223', '0224', '0225', '0226', '0227', '0228', '0229', '0231', '0232', '0233', '0234', '0235', '0236', '0237', '0238', '0239', '0240', '0241', '0242', '0243', '0248', '0249', '0250', '0251', '0252', '0253', '0254', '0255', '0256', '0257', '0258', '0260', '0261', '0262', '0263', '0264', '0265', '0266', '0267', '0268', '0269', '0270', '0271', '0272', '0273',

In [2]:
fits_list = []
#nights = ["1040","1041","1042","1043","1044","1045","1046","1047","1048","1049"]
#nights = noce[-1]
nights = ["1056",]

for night in nights:
    for f in os.listdir(path+"/"+night):
        if os.path.isfile(path+"/"+night+"/"+f):
            fits_list.append(path+"/"+night+"/"+f)

fits_list = sorted(fits_list)
print(len(fits_list))

1102


In [3]:
import ephem
from datetime import datetime

def moon_phase_pyephem(header):
    """
    Zwraca fazę Księżyca w % dla headera FITS.
    """
    obs = ephem.Observer()
    obs.lat = str(header["OBS-LAT"])
    obs.lon = str(header["OBS-LONG"])
    obs.elevation = header["OBS-ELEV"]

    # konwersja daty do formatu pyephem
    obs.date = datetime.fromisoformat(header["DATE-OBS"]).strftime("%Y/%m/%d %H:%M:%S")

    moon = ephem.Moon(obs)
    return moon.phase


In [4]:
import pickle
import warnings
from astropy.coordinates.baseframe import NonRotationTransformationWarning
warnings.filterwarnings("ignore",category=NonRotationTransformationWarning)


result = []

for f in fits_list:

    hdul = fits.open(f)
    data = hdul[0].data
    hdr = hdul[0].header
    hdul.close()

    if hdr["IMAGETYP"] == "science":
        location = EarthLocation(lat=hdr["OBS-LAT"] * u.deg, lon=hdr["OBS-LONG"] * u.deg, height=hdr["OBS-ELEV"] * u.m)
        obs_time = Time(hdr["DATE-OBS"], format="isot", scale="utc")
        field = SkyCoord(ra=hdr["RA_TEL"] * u.deg, dec=hdr["DEC_TEL"] * u.deg, frame="icrs")

        sun = get_sun(obs_time)
        moon = get_body("moon", obs_time, location)

        sun_sep = field.separation(sun)
        moon_sep = field.separation(moon)

        altaz_frame = AltAz(obstime=obs_time, location=location)
        sun_altaz = sun.transform_to(altaz_frame)
        moon_altaz = moon.transform_to(altaz_frame)


        moon_ph = moon_phase_pyephem(hdr)



        ffs = FFS(data)
        ffs.saturation = 50000
        ffs.mk_stats()
        ffs.sky_gradient()
        ffs.find_stars(threshold=5,kernel_size=10,fwhm=3)
        maska = data > np.median(data) + 3 * mad_std(data)
        line_val, line_tht, line_r = ffs.hough_transform(maska)
        fw,fwx,fwy,ell,th,cpe_t = ffs.star_info(box=15,N_stars=20)


        dqc = {}
        dqc["name"] = f
        dqc["hdr"] = hdr
        dqc["stats"] = ffs.stats
        dqc["sun_sep"] = sun_sep.deg
        dqc["moon_sep"] = moon_sep.deg
        dqc["sun_alt"] = sun_altaz.alt.deg
        dqc["moon_alt"] = moon_altaz.alt.deg
        dqc["moon_ph"] = moon_ph
        result.append(dqc)

        #print(f,len(result))

        #print(f,ffs.stats["stars"].keys())
        print(f, hdr["OBJECT"], np.nanmedian(ffs.stats["stars"]["fwhm"]))
        #print(diff_n_stars, diff_fwhm, diff_ell,diff_nc)


#with open("/home/mgorski/mglab/data/ffs_wyniki.pkl", "wb") as f:
#    pickle.dump(result, f)

# with open("/home/mgorski/mglab/data/ffs_wyniki_1050.pkl", "wb") as f:
#     pickle.dump(result, f)


TypeError: cannot unpack non-iterable NoneType object

846


In [23]:
with open("/home/mgorski/mglab/data/ffs_wyniki_1056.pkl", "wb") as f:
    pickle.dump(result, f)

In [1]:
print(ffs.stats_description)

NameError: name 'ffs' is not defined